In [1]:
 !nvidia-smi

Thu Apr 14 08:26:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

2.8.0
2.8.0


Loading the Dataset

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from fastai.vision import *
from fastai.metrics import error_rate

In [8]:
bs = 32  #batch size: if your GPU is running out of memory, set a smaller batch size, i.e 16
sz = 300 #image size
#x1 = pd.read_csv("/content/drive/MyDrive/ADHD/Dataset/Train/train_channel_1.csv", header = None).values #trainer matrix
PATH = '/content/drive/MyDrive/Image Segmentation Leaf/Rice Leaf Disease Images/'

In [10]:
classes = []
for d in os.listdir(PATH):
    if os.path.isdir(os.path.join(PATH, d)) and not d.startswith('.'):
        classes.append(d) 
print ("There are ", len(classes), "classes:\n", classes) 

There are  5 classes:
 ['Bacterialblight', 'Tungro', 'Brownspot', 'Blast', 'models']


In [11]:
for c in classes:
    print ("Class:", c)
    verify_images(os.path.join(PATH, c), delete=True);

Class: Bacterialblight


Class: Tungro


Class: Brownspot


Class: Blast


Class: models


In [12]:
data  = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(), size=sz, bs=bs, valid_pct=0.3).normalize(imagenet_stats)

/usr/local/lib/python3.7/dist-packages/fastai/vision/transform.py:247: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:766.)
  return _solve_func(B,A)[0][:,0]


In [13]:
print ("There are", len(data.train_ds), "training images and", len(data.valid_ds), "validation images." )

There are 4153 training images and 1779 validation images.


In [ ]:
data.show_batch(rows=2, figsize=(6,6))

In [ ]:
data


In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

In [ ]:
learn.lr_find();
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-2,1e-1))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
len(classes)

In [ ]:
interp.plot_top_losses(9, figsize=(22,6), heatmap=False)